In [10]:
import pandas as pd
import itertools
import json
import os
import glob
from tqdm import tqdm

In [11]:
path = 'data/다양한_형태의_한글문자_OCR/Training/form/**/*.json'
file_list = glob.glob(path, recursive=True)
train_files = [file for file in file_list if file.endswith('.json')]

In [12]:
path = 'data/다양한_형태의_한글문자_OCR/Validation/form/**/*.json'
file_list = glob.glob(path, recursive=True)
test_files = [file for file in file_list if file.endswith('.json')]

In [28]:
def getter(path_ls=[]):
    img_masks = {}
    for path in path_ls:
        img_masks_sub = {}
        with open(path) as f:
            js = json.loads(f.read())

        img_masks_sub[js['image']['file_name']] = []
        for idx, wordboxs in enumerate(js['text']['word']):
            wordbox = wordboxs['wordbox']
            value = wordboxs['value']
            img_masks_sub[js['image']['file_name']].append({
                "text" : value,
                "attribute" : {"class" : "word"},
                "bbox" : wordbox
            })
        img_masks.update(img_masks_sub)
            
    
    return img_masks

In [37]:
train_result = getter(train_files)

In [38]:
validation_range = len(test_files) // 2
validation_range

3750

In [39]:
validation_result = getter(test_files[:validation_range])

In [29]:
test_result = getter(test_files[validation_range:])

In [40]:
len(train_result), len(validation_result), len(test_result)

(60000, 3750, 3750)

In [41]:
with open('./data/train_result.json', 'w') as file:
    json.dump(train_result, file)
with open('./data/validation_result.json', 'w') as file:
    json.dump(validation_result, file)
with open('./data/test_result.json', 'w') as file:
    json.dump(test_result, file)

In [43]:
## Make gt_xxx.txt files
data_root_path = f'./data/'
save_root_path = f'./data/'

obj_list = ['test', 'train', 'validation']
for obj in obj_list:
    total_annotations = json.load(open(f'./data/{obj}_result.json'))
    gt_file = open(f'{save_root_path}gt_{obj}.txt', 'w')
    for file_name in tqdm(total_annotations):
        annotations = total_annotations[file_name]
        for idx, annotation in enumerate(annotations):
            text = annotation['text']
            gt_file.write(f'{obj}/{file_name}\t{text}\n')

100%|████████████████████████████████████| 3750/3750 [00:00<00:00, 20437.21it/s]
